Dataset on antimicrobial resistance (AMR) genes and  associated characteristics

**1.Loading Dataset**

In [ ]:
import pandas as pd
import numpy as np
data=pd.read_excel('/content/CB_combined_kmer_frequencies.xlsx')
print(data)

           ORF_ID                                               Gene  \
0     NZ_CP070943                                        AAC(6')-Ian   
1     NZ_CP070943                                               evgS   
2     NZ_CP070943                                              optrA   
3     NZ_CP070943                                               arlR   
4     NZ_CP070943                                               arlS   
...           ...                                                ...   
1909     CP063965  Agrobacterium fabrum chloramphenicol acetyltra...   
1910     CP063965                          vanR gene in vanE cluster   
1911     CP063965                                              MSI-1   
1912     CP063965                                               RanA   
1913     CP063965                                                TxR   

                      Drug Class          Resistance Mechanism  \
0      aminoglycoside antibiotic       antibiotic inactivation   
1  

**2.Checking missing values**

In [ ]:
data.isnull().sum().sum()

0

In [ ]:
data.dtypes

,0
ORF_ID,object
Gene,object
Drug Class,object
Resistance Mechanism,object
Source_File,object
...,...
CACGCC,int64
CCGACG,int64
CCCGGA,int64
CCGGAC,int64


**3.Feature Selection (Chi-square test)**

In [ ]:
X = data.drop(['ORF_ID','Gene', 'Drug Class', 'Resistance Mechanism','Source_File'], axis=1)  # Drop the columns from features
y = data[['Gene', 'Drug Class', 'Resistance Mechanism']]  # Keep these columns as the target

In [ ]:
#Discretize Continuous Features into categorical bins
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer


discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')

X_discretized = discretizer.fit_transform(X)


X_discretized = pd.DataFrame(X_discretized, columns=X.columns)

print(X_discretized.head())


   ATGCTT  TGCTTA  GCTTAA  CTTAAA  TTAAAG  TAAAGG  AAAGGG  AAGGGG  AGGGGA  \
0     0.0     1.0     1.0     0.0     0.0     0.0     0.0     1.0     2.0   
1     0.0     0.0     1.0     0.0     0.0     0.0     0.0     0.0     1.0   
2     0.0     0.0     0.0     1.0     1.0     1.0     0.0     0.0     0.0   
3     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     2.0   

   GGGGAA  ...  TGCCCG  CCCGAG  CGTGAC  GTGACC  CGTCGG  CACGCC  CCGACG  \
0     1.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1     1.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2     1.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4     1.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

   CCCGGA  CCGGAC  CGTGCC  
0     0.0     0.0     0.0  
1     0.0     0.0     0.0  
2     0.

**Normalization(min-max scaler)**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()


X_normalized = scaler.fit_transform(X_discretized)


X_normalized = pd.DataFrame(X_normalized, columns=X_discretized.columns)

print(X_normalized.head())


   ATGCTT  TGCTTA  GCTTAA  CTTAAA  TTAAAG  TAAAGG  AAAGGG  AAGGGG  AGGGGA  \
0     0.0     0.5     0.5     0.0     0.0     0.0     0.0     0.5     1.0   
1     0.0     0.0     0.5     0.0     0.0     0.0     0.0     0.0     0.5   
2     0.0     0.0     0.0     0.5     0.5     0.5     0.0     0.0     0.0   
3     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     1.0   

   GGGGAA  ...  TGCCCG  CCCGAG  CGTGAC  GTGACC  CGTCGG  CACGCC  CCGACG  \
0     0.5  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1     0.5  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2     0.5  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4     0.5  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

   CCCGGA  CCGGAC  CGTGCC  
0     0.0     0.0     0.0  
1     0.0     0.0     0.0  
2     0.

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder

for target_column in y.columns:

    y_single = y[target_column]


    encoder = LabelEncoder()
    y_encoded = encoder.fit_transform(y_single)

    chi2_values, p_values = chi2(X_encoded, y_encoded)


    chi2_results = pd.DataFrame({
        'Feature': X_encoded.columns,
        'Chi-Square': chi2_values,
        'P-Value': p_values
    })

    print(f"Chi-Square results for target: {target_column}")
    print(chi2_results)
    print("-" * 50)

Chi-Square results for target: Gene
     Feature   Chi-Square        P-Value
0     ATGCTT   848.676766   2.747888e-94
1     TGCTTA   701.774229   6.895568e-69
2     GCTTAA   725.158202   7.609989e-73
3     CTTAAA   848.500394   2.952483e-94
4     TTAAAG   657.603089   1.615232e-61
...      ...          ...            ...
4013  CACGCC   237.250000   7.037624e-05
4014  CCGACG    68.888889   1.000000e+00
4015  CCCGGA  1912.000000  2.272086e-297
4016  CCGGAC  1912.000000  2.272086e-297
4017  CGTGCC    41.500000   1.000000e+00

[4018 rows x 3 columns]
--------------------------------------------------
Chi-Square results for target: Drug Class
     Feature  Chi-Square       P-Value
0     ATGCTT  175.469680  1.471239e-24
1     TGCTTA  181.705982  9.676762e-26
2     GCTTAA  102.084356  2.789323e-11
3     CTTAAA  245.180821  4.820259e-38
4     TTAAAG  258.725301  1.018998e-40
...      ...         ...           ...
4013  CACGCC   22.538462  6.044945e-01
4014  CCGACG   22.538462  6.044945e-01
401

# Imbalance Handling(ROSE)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
import seaborn as sns
import matplotlib.pyplot as plt


ros = RandomOverSampler(sampling_strategy='auto', random_state=42)


X_balanced, y_balanced = ros.fit_resample(X_train, y_train)



# Ensemble Models

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

**Logistic Regression**

In [ ]:
#Logistic Regression

from sklearn.metrics import classification_report


log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Predict and evaluate
y_pred_log_reg = log_reg.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log_reg))


# Logistic Regression
print("Logistic Regression Classification Report:\n", classification_report(y_test, y_pred_log_reg))

Logistic Regression Accuracy: 0.97911227154047
Logistic Regression Classification Report:
                                                         precision    recall  f1-score   support

                                             APH(4)-Ia       1.00      1.00      1.00         2
                          Acinetobacter baumannii AmvA       1.00      1.00      1.00         1
Agrobacterium fabrum chloramphenicol acetyltransferase       1.00      1.00      1.00         1
                                                 CAR-1       1.00      1.00      1.00         2
                                                CRD3-1       1.00      1.00      1.00         2
                                                   CRP       1.00      1.00      1.00         4
                          Clostridium perfringens mprF       1.00      1.00      1.00         1
                           Clostridium perfringes cplR       1.00      1.00      1.00         2
                                            

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Decision Tree**

In [ ]:

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

# Predict and evaluate
y_pred_decision_tree = decision_tree.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_decision_tree))

# Decision Tree
print("Decision Tree Classification Report:\n", classification_report(y_test, y_pred_decision_tree))

Decision Tree Accuracy: 0.9686684073107049
Decision Tree Classification Report:
                                                         precision    recall  f1-score   support

                                             APH(4)-Ia       1.00      1.00      1.00         2
                          Acinetobacter baumannii AmvA       1.00      1.00      1.00         1
Agrobacterium fabrum chloramphenicol acetyltransferase       1.00      1.00      1.00         1
                                                 CAR-1       1.00      1.00      1.00         2
                                                CRD3-1       1.00      1.00      1.00         2
                                                   CRP       1.00      1.00      1.00         4
                          Clostridium perfringens mprF       1.00      1.00      1.00         1
                           Clostridium perfringes cplR       1.00      1.00      1.00         2
                                                 DHA-3

** Random Forest**

In [ ]:
#Random Forest
random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)

# Predict and evaluate
y_pred_random_forest = random_forest.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_random_forest))


# Random Forest
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_random_forest))

Random Forest Accuracy: 0.9895561357702349
Random Forest Classification Report:
                                                         precision    recall  f1-score   support

                                             APH(4)-Ia       1.00      1.00      1.00         2
                          Acinetobacter baumannii AmvA       1.00      1.00      1.00         1
Agrobacterium fabrum chloramphenicol acetyltransferase       1.00      1.00      1.00         1
                                                 CAR-1       1.00      1.00      1.00         2
                                                CRD3-1       1.00      1.00      1.00         2
                                                   CRP       1.00      1.00      1.00         4
                          Clostridium perfringens mprF       1.00      1.00      1.00         1
                           Clostridium perfringes cplR       1.00      1.00      1.00         2
                                                 DHA-3

**Feature Importance**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np


rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)

importances = rf.feature_importances_

indices = np.argsort(importances)[::-1]


N = 10
print("Top {} important features:".format(N))
for i in range(N):
    print(f"{np.array(X_train.columns)[indices[i]]}: {importances[indices[i]]}")


Top 10 important features:
TGAACC: 0.0025571453505861944
GATAAT: 0.0018367081155563664
AATATT: 0.0017169843214162948
TTTTAT: 0.0016442585343807753
AATTTA: 0.0016234333133101938
TTATTT: 0.0015679981241281688
TAATAA: 0.001515505731584598
TATATT: 0.0015014340661332933
AAACAA: 0.0014842670047928227
GAAGAA: 0.0014719777184800773


**SVM**

In [ ]:
#Support Vector Machine
svm = SVC()
svm.fit(X_train, y_train)

# Predict and evaluate
y_pred_svm = svm.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))

# SVM
print("SVM Classification Report:\n", classification_report(y_test, y_pred_svm))

SVM Accuracy: 0.9164490861618799
SVM Classification Report:
                                                         precision    recall  f1-score   support

                                             APH(4)-Ia       1.00      1.00      1.00         2
                          Acinetobacter baumannii AmvA       1.00      1.00      1.00         1
Agrobacterium fabrum chloramphenicol acetyltransferase       0.00      0.00      0.00         1
                                                 CAR-1       1.00      1.00      1.00         2
                                                CRD3-1       1.00      1.00      1.00         2
                                                   CRP       1.00      0.75      0.86         4
                          Clostridium perfringens mprF       1.00      1.00      1.00         1
                           Clostridium perfringes cplR       1.00      1.00      1.00         2
                                                 DHA-3       1.00      1.0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


knn = KNeighborsClassifier(n_neighbors=5)

# Fit the model
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"KNN Accuracy: {accuracy}")

# Classification Report
print("KNN Classification Report:")
print(classification_report(y_test, y_pred))


KNN Accuracy: 0.793733681462141
KNN Classification Report:
                                                        precision    recall  f1-score   support

                                           AAC(6')-Ian       0.00      0.00      0.00         0
                                             APH(4)-Ia       1.00      1.00      1.00         2
                          Acinetobacter baumannii AmvA       1.00      1.00      1.00         1
Agrobacterium fabrum chloramphenicol acetyltransferase       0.00      0.00      0.00         1
                                                 CAR-1       1.00      1.00      1.00         2
                                                CRD3-1       1.00      0.50      0.67         2
                                                   CRP       1.00      0.75      0.86         4
                          Clostridium perfringens mprF       0.00      0.00      0.00         1
                           Clostridium perfringes cplR       1.00      1.00 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

# K-FOLD Cross validation

In [ ]:
#K-Fold Cross Validation
#for finding best machine
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Define the models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Classifier": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

In [ ]:
# Define KFold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
# Function to print evaluation metrics
def print_evaluation_metrics(y_true, y_pred, model_name):
    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
    print("Classification Report:")
    print(classification_report(y_true, y_pred))
    print("-" * 80)


In [ ]:

if isinstance(y, pd.DataFrame):
    y = y.iloc[:, 0]

print(f"Shape of y: {y.shape}")


for name, model in models.items():
    accuracies = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)


        accuracies.append(accuracy_score(y_test, y_pred))
        print_evaluation_metrics(y_test, y_pred, name)


    print(f"Average Accuracy for {name}: {np.mean(accuracies)}\n\n")


Shape of y: (1914,)
Model: KNN
Accuracy: 0.783289817232376
Classification Report:
                                                        precision    recall  f1-score   support

                                           AAC(6')-Ian       0.00      0.00      0.00         0
                                             APH(4)-Ia       1.00      1.00      1.00         2
                          Acinetobacter baumannii AmvA       1.00      1.00      1.00         1
Agrobacterium fabrum chloramphenicol acetyltransferase       0.00      0.00      0.00         1
                                                 CAR-1       1.00      1.00      1.00         2
                                                CRD3-1       1.00      0.50      0.67         2
                                                   CRP       1.00      0.75      0.86         4
                          Clostridium perfringens mprF       0.00      0.00      0.00         1
                           Clostridium perfringes cpl

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

Model: KNN
Accuracy: 0.741514360313316
Classification Report:
                                                 precision    recall  f1-score   support

                                    AAC(6')-Ian       0.60      1.00      0.75         3
                   Acinetobacter baumannii AmvA       1.00      1.00      1.00         2
                                           BahA       1.00      1.00      1.00         3
                                          CAR-1       1.00      1.00      1.00         2
                                          CGA-1       1.00      1.00      1.00         1
                                         CRD3-1       1.00      1.00      1.00         2
                                            CRP       1.00      0.83      0.91         6
                    Clostridium perfringes cplR       1.00      1.00      1.00         2
                    Clostridium sporogenes cplR       1.00      1.00      1.00         2
                                          ELM-1

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

Model: KNN
Accuracy: 0.7911227154046997
Classification Report:
                                                 precision    recall  f1-score   support

                                    AAC(6')-Ian       0.50      1.00      0.67         1
                   Acinetobacter baumannii AmvA       1.00      1.00      1.00         3
                                          CAR-1       1.00      1.00      1.00         3
                                          CGA-1       1.00      1.00      1.00         2
                                         CRD3-1       1.00      0.67      0.80         3
                                            CRP       1.00      1.00      1.00         3
                    Clostridium perfringes cplR       1.00      1.00      1.00         1
                    Clostridium sporogenes cplR       1.00      1.00      1.00         1
                                           EdeQ       0.00      0.00      0.00         0
                                        Erm(49

...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model: SVM
Accuracy: 0.8877284595300261
Classification Report:
                                                        precision    recall  f1-score   support

                                           AAC(6')-Ian       1.00      1.00      1.00         1
                                             APH(4)-Ia       1.00      1.00      1.00         1
Agrobacterium fabrum chloramphenicol acetyltransferase       0.00      0.00      0.00         1
                                                  BahA       1.00      1.00      1.00         1
                                                 CAR-1       1.00      1.00      1.00         3
                                                 CGA-1       1.00      1.00      1.00         2
                                                CRD3-1       1.00      1.00      1.00         1
                                                   CRP       1.00      1.00      1.00         1
                           Clostridium perfringes cplR       1.00      1

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model: SVM
Accuracy: 0.8926701570680629
Classification Report:
                                                 precision    recall  f1-score   support

                                    AAC(6')-Ian       1.00      1.00      1.00         1
                                      APH(4)-Ia       1.00      1.00      1.00         3
                                           BahA       1.00      1.00      1.00         2
                                          CAR-1       1.00      1.00      1.00         2
                                          CGA-1       1.00      1.00      1.00         1
                   Clostridium perfringens mprF       1.00      1.00      1.00         1
                    Clostridium sporogenes cplR       1.00      1.00      1.00         1
                                          DHA-3       1.00      1.00      1.00         2
                                           EdeQ       1.00      1.00      1.00         1
                                        Erm(49

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
